In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import joblib
from True_Model import true_model
from copy import deepcopy
import training_function as tp
import build_new_data as bnd
import assortment as at
import warnings
import torch
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from collections import Counter
from  new_transformer_combination import Modifiled_Transformer

## Generating the new traing data or testing data

In [3]:
choice_size = 30
order_num = 10000
file_name = "test"

next_train_data = bnd.generate_data(order_num,int(choice_size))
print(next_train_data.shape)
next_train_data.to_csv(file_name+"_dataset\\"+str(order_num)+'.csv',index=False)

pro = true_model.predict(next_train_data).detach().numpy()
pro = pro[:,0:choice_size]
#pro[:,-1] = 1 - np.sum(pro[:,0:choice_size],axis = 1)
label = np.zeros(shape = pro.shape)
for i in range(pro.shape[0]):
    if np.random.binomial(1, np.sum(pro[i]), 1) > 0:
        index = np.argmax(np.random.multinomial(1, pro[i]/np.sum(pro[i]), 1))
        label[i,index] = 1
label = label[:,0:choice_size]
pro = pro[:,0:choice_size]



del_feature = ['srch_id','booking_bool']
features = [i for i in next_train_data.columns if i not in del_feature]
data_np = np.array(next_train_data[features])
data_np_cate = np.zeros(shape = data_np.shape)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np_cate[:,idx] = data_np[:,idx]
        data_np[:,idx] = 1
scaler = MinMaxScaler().fit(data_np)
data_np = scaler.transform(data_np)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np[:,idx] = 1

data_np = data_np.reshape(order_num,int(choice_size),-1)
data_np_cate = data_np_cate.reshape(order_num,int(choice_size),-1)
total_train_data = np.array([data_np_cate,data_np])


np.save(file_name+"_dataset\\data_"+'_'+str(order_num)+'.npy',total_train_data)
np.save(file_name+"_dataset\\label_"+'_'+str(order_num)+'.npy',label)
np.save(file_name+"_dataset\\pro_"+'_'+str(order_num)+'.npy',pro)
print(total_train_data.shape)

data_temp = np.zeros((order_num, choice_size, 6))
data_cate_temp = np.zeros((order_num, choice_size, 4))
for i in range(order_num):
    min1 = np.min(total_train_data[1,i,:,5])
    max1 = np.max(total_train_data[1,i,:,5])
    mean1 = np.mean(total_train_data[1,i,:,5])
    min2 = np.min(total_train_data[1,i,:,6])
    max2 = np.max(total_train_data[1,i,:,6])
    mean2 = np.mean(total_train_data[1,i,:,6])
    data_temp[i,:,0] = min1
    data_temp[i,:,1] = max1
    data_temp[i,:,2] = mean1
    data_temp[i,:,3] = min2
    data_temp[i,:,4] = max2
    data_temp[i,:,5] = mean2
    for j in range(choice_size):
        if total_train_data[1,i,j,5] == np.min(total_train_data[1,i,:,5]):
            data_cate_temp[i,j,0] = 1
        if total_train_data[1,i,j,5] == np.max(total_train_data[1,i,:,5]):
            data_cate_temp[i,j,1] = 1
        if total_train_data[1,i,j,6] == np.min(total_train_data[1,i,:,6]):
            data_cate_temp[i,j,2] = 1
        if total_train_data[1,i,j,6] == np.max(total_train_data[1,i,:,6]):
            data_cate_temp[i,j,3] = 1
data_with_assort_info = np.array([np.concatenate([total_train_data[0],np.zeros((order_num, choice_size, 6)),data_cate_temp],axis = 2),np.concatenate([total_train_data[1],data_temp,np.ones((order_num,choice_size,4))],axis = 2)])
print(data_with_assort_info.shape)
np.save(file_name+'_dataset\\data_with_assort_info_'+str(order_num )+'.npy',data_with_assort_info)

next_train_data["booking_bool"] = label.flatten()
next_train_data.to_csv(file_name + "_dataset_for_transformer\\{}.csv".format(order_num),index = False)

(300000, 13)
(2, 10000, 30, 12)
(2, 10000, 30, 22)


## Generating the decision-test data set

In [4]:
next_valid_data = bnd.generate_data(1000,100)
del_feature = ['srch_id','booking_bool']
features = [i for i in next_valid_data.columns if i not in del_feature]

data_np = np.array(next_valid_data[features])
data_np_cate = np.zeros(shape = data_np.shape)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np_cate[:,idx] = data_np[:,idx]
        data_np[:,idx] = 1
data_np = scaler.transform(data_np)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np[:,idx] = 1

data_np = data_np.reshape(1000,100,-1)
data_np_cate = data_np_cate.reshape(1000,100,-1)
valid_data = np.array([data_np_cate,data_np])
print(valid_data.shape )
next_valid_data.to_csv('new_products\\next_valid_data'+'.csv',index=False)
np.save('new_products\\valid_data_'+'.npy',valid_data)

(2, 1000, 100, 12)


## 1 MNL-Based Choice Model

### 1.1 Training the MNL model

In [ ]:
PATH = "trained_models\\MNL_parameters_"+str(order_num)+".pt"
model_MNL = tp.train_data(data,label,NUM_EPOCHS = 100,BATCH_SIZE = 32,LR = 0.01,path = PATH,feature_column=feature_column)

### 1.2 Prediction Results

In [7]:
PATH = "trained_models//MNL_parameters_"+str(order_num)+".pt"
model_MNL = torch.load(PATH)

X = torch.from_numpy(test_data[0][:,:,feature_column]).to(torch.long)
weight = torch.from_numpy(test_data[1][:,:,feature_column]).to(torch.float)
utility_MNL,u0 = model_MNL([X,weight])
utility_MNL = utility_MNL.detach().numpy()
u0 = float(u0[0])
pro_MNL = np.zeros(shape = utility_MNL.shape)
for i in range(pro_MNL.shape[0]):
    pro_MNL[i] = np.exp(utility_MNL[i])/(np.sum(np.exp(utility_MNL[i]))+np.exp(u0))

ratio = 0
high_low_ratio = []
for i in range(pro_MNL.shape[0]):
    ratio = ratio + (np.max(pro_MNL[i])- np.min(pro_MNL[i]))/np.mean(pro_MNL[i])
    #index_array = np.argsort(-test_price[i])
    #high_low_ratio.append(pro_MNL[i,index_array[1]]/pro_MNL[i,index_array[0]])

rmse = []
count_number1 = 0
count_number3 = 0
for i in range(pro_MNL.shape[0]):
    rmse.append(np.sum((pro_MNL[i] - pro[i,0:30])**2) + (np.sum(pro_MNL[i]) - np.sum(pro[i,0:30]))**2)
    pro_index = np.argsort(-pro_MNL[i])
    if np.sum(test_label[i,:]) > 0:
        if np.argmax(test_label[i,:]) in list(pro_index[0:1]):
            count_number1 += 1
        if np.argmax(test_label[i,:]) in list(pro_index[0:3]):
            count_number3 += 1
print(" MNL Top1-acc: {:.4}".format(count_number1/np.sum(test_label)))
print(" MNL Top3-acc: {:.4}".format(count_number3/np.sum(test_label)))
print(" Averaged raito of MNL: {}".format(ratio/10000))
print("RMSE of MNL: {:.4}".format(np.sqrt(sum(rmse)/(10000*31))))

 MNL Top1-acc: 0.2091
 MNL Top3-acc: 0.4363
 Averaged raito of MNL: 5.939736748902371
RMSE of MNL: 0.0217


### 1.3 Assortment Results

In [8]:
X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_MNL,u0 = model_MNL([X,weight])
utility_MNL = utility_MNL.detach().numpy()
u0 = float(u0[0])

for cadinality in [30]:
    print("{}:".format(cadinality))
    warnings.filterwarnings('ignore')
    assortment_MNL = []
    for i in tqdm(range(1000)):
        assortment_MNL.append(at.MNL(np.exp(utility_MNL[i,0:100]-u0),price[i,0:100],capacity = cadinality))
    products = []
    price_MNL = []
    for idx,item in enumerate(assortment_MNL):
        temp_price = []
        for j in range(utility_MNL.shape[1]):
            if j in item:
                products.append(idx*utility_MNL.shape[1]+j)
                temp_price.append(price[idx,j])
        price_MNL.append(temp_price)
    valid_pro = true_model.predict(next_valid_data.loc[products]).detach().numpy()
    print(np.sum(valid_pro[3][0:10]))
    total_profit = 0
    for i in range(len(price_MNL)):
        total_profit += np.dot(valid_pro[i,0:len(price_MNL[i])],np.array(price_MNL[i]))
    print(total_profit/1000)

    assortment_array = np.zeros((1000,100))
    for i,item in enumerate(assortment_MNL):
        for j in item:
            assortment_array[i,j] = 1
    np.save("Assortment//MNL_assortment_"+str(order_num )+'_'+str(cadinality)+".npy",assortment_array)
    a = np.sum(assortment_array,axis = 1)
    print(np.min(a),np.mean(a),np.max(a))

## 2 DeepFM-based Choice Model

### 2.1 Training the DeepFM model

In [12]:
total = data.shape[1]
indices = np.random.permutation(total)
k_fold = 5
training_data = data[:,indices[0:int(total*(k_fold-1)/k_fold)],:,:]
training_label = label[indices[0:int(total*(k_fold-1)/k_fold)],:]
training_valid_data = data[:,indices[int(total*(k_fold-1)/k_fold):],:,:]
training_valid_label = label[indices[int(total*(k_fold-1)/k_fold):],:]
weight = int((training_label.shape[0]*training_label.shape[1]-np.sum(training_label))/np.sum(training_label))

In [ ]:
PATH = 'trained_models//DeepFM_parameters_'+str(order_num )+'.pt'
model_DeepFM = tp.train_DeepFM(training_data,training_label,training_valid_data,training_valid_label,weight1 = weight,path = PATH,LR = 0.05,feature_column=feature_column)

### 2.2 Prediction Results

In [14]:
PATH = 'trained_models//DeepFM_parameters_'+str(order_num )+'.pt'
model_DeepFM = torch.load(PATH)
purchase_pro = np.sum(label)/label.shape[0]

X = torch.from_numpy(test_data[0][:,:,feature_column]).to(torch.long)
weight = torch.from_numpy(test_data[1][:,:,feature_column]).to(torch.float)
utility_DeepFM = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()

ratio = 0
high_low_ratio = []
for i in range(pro_DeepFM.shape[0]):
    ratio = ratio + (np.max(pro_DeepFM[i])- np.min(pro_DeepFM[i]))/np.mean(pro_DeepFM[i])
    # index_array = np.argsort(-valid_price[i])
    # high_low_ratio.append(pro_DeepFM[i,index_array[1]]/pro_DeepFM[i,index_array[0]])

rmse = []
count_number1 = 0
count_number3 = 0
for i in range(pro_DeepFM.shape[0]):
    temp_pro = purchase_pro*pro_DeepFM[i]/np.sum(pro_DeepFM[i])
    rmse.append(np.sum((temp_pro - pro[i,0:30])**2) + (np.sum(temp_pro) - np.sum(pro[i,0:30]))**2)
    pro_index = np.argsort(-pro_DeepFM[i])
    if np.sum(test_label[i,:]) > 0:
        if np.argmax(test_label[i,:]) in list(pro_index[0:1]):
            count_number1 += 1
        if np.argmax(test_label[i,:]) in list(pro_index[0:3]):
            count_number3 += 1
print(" DeepFM Top1-acc: {:.4}".format(count_number1/np.sum(test_label)))
print(" DeepFM Top3-acc: {:.4}".format(count_number3/np.sum(test_label)))
print(" Averaged raito of DeepFM: {}".format(ratio/10000))
print("RMSE of DeepFM: {:.4}".format(np.sqrt(sum(rmse)/(10000*31))))

 DeepFM Top1-acc: 0.2095
 DeepFM Top3-acc: 0.4412
 Averaged raito of DeepFM: 2.0982225510954855
RMSE of DeepFM: 0.03312


### 2.3 Assortment Results

In [15]:
X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_DeepFM  = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()
for cadinality in [30]:
    warnings.filterwarnings('ignore')
    assortment_DeepFM = []
    for i in range(pro_DeepFM.shape[0]):
        #length = len(assortment_ml[i])
        revenue = pro_DeepFM[i,0:100]*price[i,0:100]
        sort_index = np.argsort(-revenue)
        assortment_DeepFM.append(sort_index[0:cadinality])

    products = []
    price_MNL = []
    for idx,item in enumerate(assortment_DeepFM):
        temp_price = []
        for j in range(utility_MNL.shape[1]):
            if j in item:
                products.append(idx*utility_MNL.shape[1]+j)
                temp_price.append(price[idx,j])
        price_MNL.append(temp_price)
    valid_pro = true_model.predict(next_valid_data.loc[products]).detach().numpy()
    total_profit = 0
    for i in range(len(price_MNL)):
        total_profit += np.dot(valid_pro[i,0:len(price_MNL[i])],np.array(price_MNL[i]))
    print(total_profit/1000)

149.82860732451886


## 3 DeepFM-a based choice model

### 3.1 Training the DeepFM-a model

In [16]:
total = data_with_assort_info.shape[1]
indices = np.random.permutation(total) 
k_fold = 5
training_data = data_with_assort_info[:,indices[0:int(total*(k_fold-1)/k_fold)],:,:]
training_label = label[indices[0:int(total*(k_fold-1)/k_fold)],:]
training_valid_data = data_with_assort_info[:,indices[int(total*(k_fold-1)/k_fold):],:,:]
training_valid_label = label[indices[int(total*(k_fold-1)/k_fold):],:]
weight = int((training_label.shape[0]*training_label.shape[1]-np.sum(training_label))/np.sum(training_label))

In [ ]:
PATH = 'trained_models//DeepFM_parameters_with_assortment'+'_'+str(order_num )+'.pt'
model_DeepFM = tp.train_DeepFM(training_data,training_label,training_valid_data,training_valid_label,model_name = 'Assortment',weight1 = weight,LR = 0.05,path = PATH,feature_column = [i for i in range(22)])

### 3.2 Prediction Results

In [20]:
PATH = 'trained_models//DeepFM_parameters_with_assortment'+'_'+str(order_num )+'.pt'
model_DeepFM = torch.load(PATH)
purchase_pro = np.sum(label)/label.shape[0]

X = torch.from_numpy(test_data_with_assort_info[0][:,:,feature_column_swapping]).to(torch.long)
weight = torch.from_numpy(test_data_with_assort_info[1][:,:,feature_column_swapping]).to(torch.float)
utility_DeepFM  = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()

ratio = 0
high_low_ratio = []
for i in range(pro_DeepFM.shape[0]):
    ratio = ratio + (np.max(pro_DeepFM[i])- np.min(pro_DeepFM[i]))/np.mean(pro_DeepFM[i])
    # index_array = np.argsort(-valid_price[i])
    # high_low_ratio.append(pro_DeepFM[i,index_array[1]]/pro_DeepFM[i,index_array[0]])

rmse = []
count_number1 = 0
count_number3 = 0
for i in range(pro_DeepFM.shape[0]):
    temp_pro = purchase_pro*pro_DeepFM[i]/np.sum(pro_DeepFM[i])
    rmse.append(np.sum((temp_pro - pro[i,0:30])**2) + (np.sum(temp_pro) - np.sum(pro[i,0:30]))**2)
    pro_index = np.argsort(-pro_DeepFM[i])
    if np.sum(test_label[i,:]) > 0:
        if np.argmax(test_label[i,:]) in list(pro_index[0:1]):
            count_number1 += 1
        if np.argmax(test_label[i,:]) in list(pro_index[0:3]):
            count_number3 += 1
print(" DeepFM-assort Top1-acc: {:.4}".format(count_number1/np.sum(test_label)))
print(" DeepFM-assort Top3-acc: {:.4}".format(count_number3/np.sum(test_label)))
print(" Averaged raito of DeepFM-assort: {}".format(ratio/10000))
print("RMSE of DeepFM-assort: {:.4}".format(np.sqrt(sum(rmse)/(10000*31))))

 DeepFM-assort Top1-acc: 0.2076
 DeepFM-assort Top3-acc: 0.4381
 Averaged raito of DeepFM-assort: 2.065047556900978
RMSE of DeepFM-assort: 0.03339


### 3.2 Assortment Results

In [ ]:
for cadinality in [30]:
    model_DeepFM1 = torch.load('trained_models//DeepFM_parameters_'+str(order_num )+'.pt')
    X = torch.from_numpy(valid_data[0]).to(torch.long)
    weight = torch.from_numpy(valid_data[1]).to(torch.float)
    utility_DeepFM  = model_DeepFM1([X,weight])
    pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()
    assortment_DeepFM = []
    for i in range(pro_DeepFM.shape[0]):
        #length = len(assortment_ml[i])
        revenue = pro_DeepFM[i,0:100]*price[i,0:100]
        sort_index = np.argsort(-revenue)
        assortment_DeepFM.append(sort_index[0:cadinality])
    warnings.filterwarnings('ignore')
    assortment_ml = []
    for i in tqdm(range(1000)):
        assortment_ml.append(at.ml_assortment_swap(model_DeepFM,valid_data[:,i,0:100,:],price[i,0:100],assortment_DeepFM[i],capacity = cadinality,feature_column = [i for i in range(22)]))
    products = []
    price_MNL = []
    for idx,item in enumerate(assortment_ml):
        temp_price = []
        for j in range(valid_data.shape[2]):
            if j in item:
                products.append(idx*valid_data.shape[2]+j)
                temp_price.append(price[idx,j])
        price_MNL.append(temp_price)
    valid_pro = true_model.predict(next_valid_data.loc[products]).detach().numpy()
    total_profit = 0
    for i in range(len(price_MNL)):
        total_profit += np.dot(valid_pro[i,0:len(price_MNL[i])],np.array(price_MNL[i]))
    assortment_array = np.zeros((1000,100))
    for i,item in enumerate(assortment_ml):
        for j in item:
            assortment_array[i,j] = 1   
    np.save("Assortment//DeepFM_assortment_swapping_"+str(order_num )+'_'+str(cadinality)+".npy",assortment_array)
    print(total_profit/1000)

## 4 Transformer

### 4.1 Train the transformer model

In [10]:
train_data = pd.read_csv('train_dataset_for_transformer//{}.csv'.format(20000))
weight_index = np.sum(train_data['booking_bool'])/(len(list(train_data['srch_id'].unique()))-np.sum(train_data['booking_bool']))
if weight_index > 1:
    pt_weight = [1,int(weight_index)]
else:
    pt_weight = [int(1/weight_index),1]
#pt_weight = [8,1]

In [ ]:
Modifiled_Transformer.train_model(train_data,pt_weight = pt_weight,model_name = "20000")

### 4.2 Obtaining the prediction accuracy

In [12]:
test_data = pd.read_csv('test_dataset_for_transformer//10000.csv')
label = np.load('test_dataset//label__10000.npy')
pro = np.load('test_dataset//pro__10000.npy')

In [13]:
pro_pred = Modifiled_Transformer.predict(test_data[test_data.columns[0:13]],max_num = 30, pt_weight = pt_weight,model_name = '20000')
pro_pred = pro_pred.detach().numpy()
total = 0
count1 = 0
count3 = 0
RMSE_t = []
ratio = 0
for i in range(pro_pred.shape[0]):
    ratio = ratio + (np.max(pro_pred[i,0:30])- np.min(pro_pred[i,0:30]))/np.mean(pro_pred[i,0:30])
for i in range(pro_pred.shape[0]):
    RMSE_t.append(np.sum((pro_pred[i,0:30]-pro[i,0:30])**2)+(np.sum(pro_pred[i,0:30]) - np.sum(pro[i,0:30]))**2)
    if np.sum(label[i,:]) == 1:
        total = total + 1
        if np.argmax(label[i,:]) in np.argsort(-pro_pred[i,:])[0:1]:
            count1 = count1 + 1
        if np.argmax(label[i,:]) in np.argsort(-pro_pred[i,:])[0:3]:
            count3 = count3 + 1
print("Top 1-Acc:  {:0.4}".format(count1/total))
print("Top 3-Acc:  {:0.4}".format(count3/total))
print("RMSE: {}".format(np.sqrt(sum(RMSE_t)/(len(RMSE_t)*31))))
print(" Averaged raito of Transformer: {}".format(ratio/10000))

Top 1-Acc:  0.2035
Top 3-Acc:  0.4351
RMSE: 0.022282694032881138
 Averaged raito of Transformer: 6.182143734765053


### 4.3 Get the assortment

In [ ]:
for cardinality in [30]:
    print(cardinality)
    warnings.filterwarnings('ignore')
    PATH = 'trained_models//DeepFM_parameters_'+str(20000 )+'.pt'
    model_DeepFM = torch.load(PATH)
    valid_data = np.load('new_products//valid_data_'+'.npy')
    next_valid_data = pd.read_csv('new_products//next_valid_data'+'.csv') 
    price = np.array(next_valid_data['price_usd']).reshape(-1,100)
    X = torch.from_numpy(valid_data[0]).to(torch.long)
    weight = torch.from_numpy(valid_data[1]).to(torch.float)
    utility_DeepFM  = model_DeepFM([X,weight])
    pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()
    assortment_DeepFM = []
    for i in range(pro_DeepFM.shape[0]):
        #length = len(assortment_ml[i])
        revenue = pro_DeepFM[i,0:100]*price[i,0:100]
        sort_index = np.argsort(-revenue)
        assortment_DeepFM.append(sort_index[0:cardinality])
    warnings.filterwarnings('ignore')

    assortment_transformer = []

    for i in tqdm(range(1000)):
        temp_df = next_valid_data[next_valid_data.srch_id == i]
        temp_df.reset_index(drop = True,inplace = True)
        temp_df = temp_df.loc[0:99,:]
        temp_price = np.array(temp_df.loc[0:99,'price_usd'])
        assortment_transformer.append(at.transformer_assortment_swap("20000",temp_df,list(assortment_DeepFM[i]),pt_weight,temp_price))
    products = []
    price_MNL = []
    for idx,item in enumerate(assortment_transformer):
        temp_price = []
        for j in range(valid_data.shape[2]):
            if j in item:
                products.append(idx*valid_data.shape[2]+j)
                temp_price.append(price[idx,j])
        price_MNL.append(temp_price)
    valid_pro = true_model.predict(next_valid_data.loc[products]).detach().numpy()
    total_profit = 0
    for i in range(len(price_MNL)):
        total_profit += np.dot(valid_pro[i,0:len(price_MNL[i])],np.array(price_MNL[i]))
    print(total_profit/1000)
    assortment_array = np.zeros((1000,100))
    for i,item in enumerate(assortment_transformer):
        for j in item:
            assortment_array[i,j] = 1   
    np.save("Assortment//Transformer_"+str(20000)+'_'+str(cardinality)+".npy",assortment_array)